In [7]:
# Import necessary libraries
import gradio as gr
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.preprocessing import StandardScaler

# Load your dataset (assuming 'crime.csv' file is available)
crime = pd.read_csv('07. crime_dataset_analysis_v2.csv')
crime = pd.DataFrame(crime)

# Create a scaler instance
scaler = StandardScaler()

# Scale the 'Victim Age' for model training
crime['Victim Age Scaled'] = scaler.fit_transform(crime[['Victim Age']])

# Prepare the model's feature data (X) and target variable (y)
X_dummy = crime.drop(columns=['Victim Age', 'Weapon Category'])
y_dummy = crime['Weapon Category']

# Initialize and train the Logistic Regression model
model = LogisticRegression()
model.fit(X_dummy, y_dummy)

# Helper function to convert "Yes"/"No" to numeric
def convert_input(value):
    return 1 if value == "Yes" else 0

# Scale victim age (same scaling as done in the dataset) for the interface
def scale_victim_age(victim_age):
    return scaler.transform([[victim_age]])[0][0]  # Scale based on the previously fitted scaler

# Function to format confusion matrix (improved clarity)
def format_confusion_matrix(matrix):
    return (
        f"Confusion Matrix:\n"
        f"               Predicted: No     Predicted: Yes\n"
        f"Actual: No    {matrix[0][0]:<6} {matrix[0][1]:<6}\n"
        f"Actual: Yes   {matrix[1][0]:<6} {matrix[1][1]:<6}"
    )

# Gradio interface function
def gradio_interface(
    region_northeast, region_south, region_west,
    season_spring, season_summer, season_winter,
    relationship_family, relationship_lover, relationship_stranger,
    agency_other, agency_sheriff,
    victim_sex_male, perpetrator_sex_male,
    ethnicity_not_hispanic, victim_age
):
    # Validate that only one "Yes" is selected for each category
    def validate_category(category, category_name):
        if category.count("Yes") > 1:
            return f"Error: Only one 'Yes' is allowed in the {category_name} category."
        return None
    
    # Check for errors in each category
    region_error = validate_category([region_northeast, region_south, region_west], "Region")
    season_error = validate_category([season_spring, season_summer, season_winter], "Season")
    relationship_error = validate_category([relationship_family, relationship_lover, relationship_stranger], "Relationship")
    agency_error = validate_category([agency_other, agency_sheriff], "Agency")

    # If there's an error, return the error message
    if region_error:
        return region_error, "", "", "", "", "", "", ""
    if season_error:
        return season_error, "", "", "", "", "", "", ""
    if relationship_error:
        return relationship_error, "", "", "", "", "", "", ""
    if agency_error:
        return agency_error, "", "", "", "", "", "", ""

    # Convert dropdown inputs ("Yes"/"No") to numeric
    input_features = [
        convert_input(region_northeast), convert_input(region_south), convert_input(region_west),
        convert_input(season_spring), convert_input(season_summer), convert_input(season_winter),
        convert_input(relationship_family), convert_input(relationship_lover), convert_input(relationship_stranger),
        convert_input(agency_other), convert_input(agency_sheriff),
        convert_input(victim_sex_male), convert_input(perpetrator_sex_male),
        convert_input(ethnicity_not_hispanic),
        scale_victim_age(victim_age),  # Scale victim age for the model
    ]

    # Model predictions
    input_pred = model.predict([input_features])[0]  # Single prediction
    input_pred_proba = model.predict_proba([input_features])[0]  # Probabilities for both classes

    # Map numeric prediction to labels
    prediction_label = "Firearm" if input_pred == 1 else "Non-Firearm"

    # Compute evaluation metrics
    y_pred = model.predict(X_dummy)
    conf_matrix = confusion_matrix(y_dummy, y_pred)
    accuracy = accuracy_score(y_dummy, y_pred)
    recall = recall_score(y_dummy, y_pred)
    precision = precision_score(y_dummy, y_pred)
    f1 = f1_score(y_dummy, y_pred)

    # Improved confusion matrix format
    formatted_conf_matrix = format_confusion_matrix(conf_matrix)

    return (
        f"Prediction: {prediction_label}",
        f"Probability of Firearm Being Used: {input_pred_proba[1]:.2f}",
        f"Probability of Firearm Not Being Used: {input_pred_proba[0]:.2f}",
        f"{formatted_conf_matrix}",
        f"Accuracy: {accuracy:.2f}",
        f"Recall: {recall:.2f}",
        f"Precision: {precision:.2f}",
        f"F1 Score: {f1:.2f}"
    )

# Gradio dropdowns for inputs
region_dropdowns = [
    gr.Dropdown(["No", "Yes"], value="No", label="Region_Northeast"),
    gr.Dropdown(["No", "Yes"], value="No", label="Region_South"),
    gr.Dropdown(["No", "Yes"], value="No", label="Region_West")
]

season_dropdowns = [
    gr.Dropdown(["No", "Yes"], value="No", label="Season_Spring"),
    gr.Dropdown(["No", "Yes"], value="No", label="Season_Summer"),
    gr.Dropdown(["No", "Yes"], value="No", label="Season_Winter")
]

relationship_dropdowns = [
    gr.Dropdown(["No", "Yes"], value="No", label="Relationship Category_Family"),
    gr.Dropdown(["No", "Yes"], value="No", label="Relationship Category_Lover"),
    gr.Dropdown(["No", "Yes"], value="No", label="Relationship Category_Stranger")
]

agency_dropdowns = [
    gr.Dropdown(["No", "Yes"], value="No", label="Agency_Type_grouped_Other Police"),
    gr.Dropdown(["No", "Yes"], value="No", label="Agency_Type_grouped_Sheriff")
]

victim_sex_dropdown = gr.Dropdown(["No", "Yes"], value="No", label="Victim Sex_Male")
perpetrator_sex_dropdown = gr.Dropdown(["No", "Yes"], value="No", label="Perpetrator Sex_Male")
ethnicity_dropdown = gr.Dropdown(["No", "Yes"], value="No", label="Ethnicity_Not_Hispanic_Combined")

victim_age_slider = gr.Slider(0, 100, step=1, value=0, label="Victim Age")

# Custom CSS for the Gradio interface
custom_css = """
body {
    background: linear-gradient(to bottom right, #6a11cb, #2575fc); /* Gradient background */
    color: white;
    font-family: Arial, sans-serif;
}
.gradio-container {
    background: rgba(180, 180, 180, 0.5); /* Semi-transparent white */
    border-radius: 15px;
    padding: 20px;
    box-shadow: 0px 4px 12px rgba(0, 0, 0, 0.2);
}
.gradio-container .input-block input, 
.gradio-container .input-block select, 
.gradio-container .input-block textarea {
    background: #e0f7fa; /* Light cyan for input boxes */
    border: 1px solid #00796b;
    border-radius: 5px;
    padding: 5px;
    color: #004d40;
}
.gradio-container .output-block textarea {
    background: #ffebee; /* Light pink for output boxes */
    border: 1px solid #d32f2f;
    border-radius: 5px;
    padding: 5px;
    color: #b71c1c;
}
"""

# Gradio interface with event handlers to update dropdowns
gr.Interface(
    title="Weapon Use Prediction",
    description="**Instructions:** Select at most one 'Yes' per category. For example, only one 'Yes' in the region, season, relationship, and agency categories.",
    fn=gradio_interface,
    inputs=region_dropdowns + season_dropdowns + relationship_dropdowns + agency_dropdowns + [victim_sex_dropdown, perpetrator_sex_dropdown, ethnicity_dropdown, victim_age_slider],
    outputs=[
        gr.Textbox(label="Prediction"),
        gr.Textbox(label="Probability of Firearm Being Used"),
        gr.Textbox(label="Probability of Firearm Not Being Used"),
        gr.Textbox(label="Confusion Matrix"),
        gr.Textbox(label="Accuracy"),
        gr.Textbox(label="Recall"),
        gr.Textbox(label="Precision"),
        gr.Textbox(label="F1 Score"),
    ],
    css=custom_css  # Applying the custom CSS to style the interface
).launch()


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
